In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Kidney_Chromophobe"
cohort = "GSE19949"

# Input paths
in_trait_dir = "../../input/GEO/Kidney_Chromophobe"
in_cohort_dir = "../../input/GEO/Kidney_Chromophobe/GSE19949"

# Output paths
out_data_file = "../../output/preprocess/Kidney_Chromophobe/GSE19949.csv"
out_gene_data_file = "../../output/preprocess/Kidney_Chromophobe/gene_data/GSE19949.csv"
out_clinical_data_file = "../../output/preprocess/Kidney_Chromophobe/clinical_data/GSE19949.csv"
json_path = "../../output/preprocess/Kidney_Chromophobe/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Integrative genome-wide expression profiling identifies three distinct molecular subgroups of renal cell carcinoma with different patient outcome"
!Series_summary	"Background: Renal cell carcinoma (RCC) is characterized by a number of diverse molecular aberrations that differ among individuals. Recent approaches to molecularly classify RCC were based on clinical, pathological as well as on single molecular parameters. As a consequence, gene expression patterns reflecting the sum of genetic aberrations in individual tumors may not have been recognized. In an attempt to uncover such molecular features in RCC, we used a novel, unbiased and integrative approach."
!Series_summary	"Methods: We integrated gene expression data from 97 primary RCCs of different pathologic parameters, 15 RCC metastases as well as 34 cancer cell lines for two-way nonsupervised hierarchical clustering using gene groups suggested by the PANTHER Classification System. We depict

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import os
import numpy as np
import json
from typing import Optional, Callable, Dict, Any

# 1. Gene Expression Data Availability
# Based on the background information, this is an "Expression profiling by array" study 
# that involves renal cell carcinoma samples. The text mentions using Affymetrix platform
# for genomic profiling, which suggests gene expression data is available.
is_gene_available = True

# 2. Variable Availability and Data Type Conversion

# 2.1 For the trait (Kidney_Chromophobe):
# Looking at row 4, we can see "icd-o 3 diagnosis text: renal cell carcinoma, chromophobe"
# which corresponds to the Kidney_Chromophobe trait
trait_row = 4

# Define conversion function for trait
def convert_trait(value):
    if pd.isna(value):
        return None
    
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Check if the value indicates chromophobe RCC
    if 'chromophobe' in value.lower():
        return 1  # Case: chromophobe RCC
    elif 'renal cell carcinoma' in value.lower() or 'rcc' in value.lower():
        return 0  # Case: other types of RCC
    else:
        return None  # Unknown or irrelevant value

# 2.2 For age:
# After reviewing the sample characteristics, there is no indication of age data
age_row = None

def convert_age(value):
    # This function is defined but won't be used as age data is not available
    return None

# 2.3 For gender:
# Gender information is available in row 6
gender_row = 6

def convert_gender(value):
    if pd.isna(value):
        return None
    
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Convert gender to binary
    if value.lower() == 'female':
        return 0
    elif value.lower() == 'male':
        return 1
    else:
        return None  # Unknown or NA

# 3. Save Metadata - Initial Filtering
# Check if trait data is available
is_trait_available = trait_row is not None
# Validate and save cohort information
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction - Only if trait_row is not None
if trait_row is not None:
    # Assuming clinical_data is available from previous steps
    try:
        # Load clinical data from file
        clinical_data = pd.DataFrame()
        clinical_data_files = [f for f in os.listdir(in_cohort_dir) if f.endswith('_clinical.txt')]
        if clinical_data_files:
            clinical_data = pd.read_csv(os.path.join(in_cohort_dir, clinical_data_files[0]), sep='\t')
        
        # Extract clinical features
        selected_clinical_df = geo_select_clinical_features(
            clinical_df=clinical_data,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        # Preview the extracted clinical features
        preview = preview_df(selected_clinical_df)
        print("Clinical Data Preview:")
        print(preview)
        
        # Create directory if it doesn't exist
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        
        # Save to CSV
        selected_clinical_df.to_csv(out_clinical_data_file, index=False)
        print(f"Clinical data saved to {out_clinical_data_file}")
    except Exception as e:
        print(f"Error in clinical feature extraction: {e}")
else:
    print("Clinical data extraction skipped as trait_row is None")


Clinical Data Preview:
{}
Clinical data saved to ../../output/preprocess/Kidney_Chromophobe/clinical_data/GSE19949.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Extract gene expression data from the matrix file
try:
    print("Extracting gene data from matrix file:")
    gene_data = get_genetic_data(matrix_file)
    if gene_data.empty:
        print("Extracted gene expression data is empty")
        is_gene_available = False
    else:
        print(f"Successfully extracted gene data with {len(gene_data.index)} rows")
        print("First 20 gene IDs:")
        print(gene_data.index[:20])
        is_gene_available = True
except Exception as e:
    print(f"Error extracting gene data: {e}")
    print("This dataset appears to have an empty or malformed gene expression matrix")
    is_gene_available = False

print(f"\nGene expression data available: {is_gene_available}")


Extracting gene data from matrix file:


Successfully extracted gene data with 22277 rows
First 20 gene IDs:
Index(['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at',
       '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at',
       '1494_f_at', '1598_g_at', '160020_at', '1729_at', '1773_at', '177_at',
       '179_at', '1861_at'],
      dtype='object', name='ID')

Gene expression data available: True


### Step 4: Gene Identifier Review

In [5]:
# These IDs are from Affymetrix probe IDs (e.g., '1007_s_at'), not human gene symbols.
# They need to be mapped to standard gene symbols for biological interpretation.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Extract gene annotation data from the SOFT file
print("Extracting gene annotation data from SOFT file...")
try:
    # Use the library function to extract gene annotation
    gene_annotation = get_gene_annotation(soft_file)
    print(f"Successfully extracted gene annotation data with {len(gene_annotation.index)} rows")
    
    # Preview the annotation DataFrame
    print("\nGene annotation preview (first few rows):")
    print(preview_df(gene_annotation))
    
    # Show column names to help identify which columns we need for mapping
    print("\nColumn names in gene annotation data:")
    print(gene_annotation.columns.tolist())
    
    # Check for relevant mapping columns
    if 'GB_ACC' in gene_annotation.columns:
        print("\nThe dataset contains GenBank accessions (GB_ACC) that could be used for gene mapping.")
        # Count non-null values in GB_ACC column
        non_null_count = gene_annotation['GB_ACC'].count()
        print(f"Number of rows with GenBank accessions: {non_null_count} out of {len(gene_annotation)}")
    
    if 'SPOT_ID' in gene_annotation.columns:
        print("\nThe dataset contains genomic regions (SPOT_ID) that could be used for location-based gene mapping.")
        print("Example SPOT_ID format:", gene_annotation['SPOT_ID'].iloc[0])
    
except Exception as e:
    print(f"Error processing gene annotation data: {e}")
    is_gene_available = False


Extracting gene annotation data from SOFT file...


Successfully extracted gene annotation data with 5177938 rows

Gene annotation preview (first few rows):
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Mar 8, 2007', 'Mar 8, 2007', 'Mar 8, 2007', 'Mar 8, 2007', 'Mar 8, 2007'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': [nan, nan, nan, nan, nan], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /DEFINITION=HSP70B Human heat-shock protein HSP70B' gene", 'X69699 /FEATURE= /DEFINITION=HSPAX8A H.sapi

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the columns for gene identifiers and gene symbols in the annotation data
# From the previewed data, we can see 'ID' column contains probe identifiers similar to the gene expression data indices
# and 'Gene Symbol' column contains the corresponding gene symbols

# 2. Create gene mapping dataframe
print("Creating gene mapping dataframe...")
try:
    mapping_df = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='Gene Symbol')
    print(f"Successfully created gene mapping with {len(mapping_df)} rows")
    print("First few rows of mapping dataframe:")
    print(mapping_df.head())
    
    # 3. Apply gene mapping to convert probe-level measurements to gene expression data
    print("\nConverting probe measurements to gene expression data...")
    gene_data = apply_gene_mapping(gene_data, mapping_df)
    print(f"Successfully converted to gene expression data with {len(gene_data)} genes")
    print("First few gene symbols:")
    print(gene_data.index[:10])
    
    # Save the gene expression data to a file
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    gene_data.to_csv(out_gene_data_file)
    print(f"Gene expression data saved to {out_gene_data_file}")
    
except Exception as e:
    print(f"Error in gene mapping process: {e}")
    is_gene_available = False

print(f"\nGene expression data available: {is_gene_available}")


Creating gene mapping dataframe...


Successfully created gene mapping with 21248 rows
First few rows of mapping dataframe:
          ID    Gene
0  1007_s_at    DDR1
1    1053_at    RFC2
2     117_at   HSPA6
3     121_at    PAX8
4  1255_g_at  GUCA1A

Converting probe measurements to gene expression data...
Successfully converted to gene expression data with 13046 genes
First few gene symbols:
Index(['A2BP1', 'A2M', 'A4GALT', 'A4GNT', 'AAAS', 'AACS', 'AADAC', 'AAK1',
       'AAMP', 'AANAT'],
      dtype='object', name='Gene')


Gene expression data saved to ../../output/preprocess/Kidney_Chromophobe/gene_data/GSE19949.csv

Gene expression data available: True


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
print("\nNormalizing gene symbols...")
try:
    normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
    print(f"After normalization: {len(normalized_gene_data.index)} unique gene symbols")
    
    # Save the normalized gene data
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    normalized_gene_data.to_csv(out_gene_data_file)
    print(f"Normalized gene expression data saved to {out_gene_data_file}")
except Exception as e:
    print(f"Error normalizing gene symbols: {e}")
    normalized_gene_data = gene_data  # Use original data if normalization fails

# 2. Extract clinical features
print("\nExtracting clinical features...")
try:
    # Examine clinical_data structure for debugging
    print("Clinical data shape:", clinical_data.shape)
    print("Clinical data columns:", clinical_data.columns[:5].tolist())
    print("Clinical data index:", clinical_data.index[:5].tolist())
    
    # Looking at the sample characteristics dictionary from Step 1
    # Row 7 contains tissue type with chromophobe RCC information
    # Row 6 contains gender information
    
    trait_row = 7
    
    def convert_trait(value):
        """Convert histology information to binary trait (Kidney_Chromophobe)"""
        if pd.isna(value):
            return None
        
        # Extract the value after the colon if present
        if ":" in value:
            value = value.split(":", 1)[1].strip()
        
        # Check if the value mentions chromophobe kidney cancer
        if "chromophobe" in value.lower():
            return 1
        elif "renal cell carcinoma" in value.lower() or "rcc" in value.lower():
            return 0
        else:
            return None
    
    gender_row = 6
    
    def convert_gender(value):
        """Convert gender information to binary (0=female, 1=male)"""
        if pd.isna(value):
            return None
        
        # Extract the value after the colon if present
        if ":" in value:
            value = value.split(":", 1)[1].strip()
        
        value = value.lower()
        if "female" in value:
            return 0
        elif "male" in value:
            return 1
        else:
            return None
    
    # Age not available in this dataset
    age_row = None
    
    def convert_age(value):
        return None
    
    # Extract clinical features
    clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    print(f"Clinical data extracted with shape: {clinical_df.shape}")
    print("Clinical data preview:")
    print(preview_df(clinical_df.T))
    
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    
    # Save clinical data
    clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")
    is_trait_available = clinical_df.shape[1] > 0  # Check if we have any samples
except Exception as e:
    print(f"Error extracting clinical features: {e}")
    is_trait_available = False
    clinical_df = pd.DataFrame()

# 3. Link clinical and genetic data
print("\nLinking clinical and genetic data...")
try:
    if not clinical_df.empty and not normalized_gene_data.empty and clinical_df.shape[1] > 0:
        # Print sample IDs from both datasets for debugging
        print("First few clinical sample columns:", list(clinical_df.columns)[:5])
        print("First few genetic sample columns:", list(normalized_gene_data.columns)[:5])
        
        # Check if there are common sample IDs
        common_samples = set(clinical_df.columns).intersection(set(normalized_gene_data.columns))
        print(f"Number of common samples: {len(common_samples)}")
        
        # Link clinical and genetic data
        linked_data = geo_link_clinical_genetic_data(clinical_df, normalized_gene_data)
        print(f"Linked data shape: {linked_data.shape}")
        
        # Check if we have at least one sample with trait value
        if trait in linked_data.columns:
            trait_count = linked_data[trait].count()
            print(f"Number of samples with trait values: {trait_count}")
            
            if trait_count > 0:
                # 4. Handle missing values systematically
                print("\nHandling missing values...")
                linked_data = handle_missing_values(linked_data, trait)
                print(f"After handling missing values, data shape: {linked_data.shape}")
                
                # Check if we still have samples
                if linked_data.shape[0] > 0:
                    # 5. Determine whether features are biased
                    print("\nChecking for bias in features...")
                    is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
                else:
                    print("No samples remain after handling missing values")
                    is_biased = True
            else:
                print("No samples have valid trait values")
                is_biased = True
        else:
            print(f"Trait column '{trait}' not found in linked data")
            is_biased = True
    else:
        print("Cannot link data: clinical or genetic data is missing/empty")
        linked_data = pd.DataFrame()
        is_biased = True
        
except Exception as e:
    print(f"Error in linking clinical and genetic data: {e}")
    linked_data = pd.DataFrame()
    is_biased = True

# 6. Final quality validation
print("\nPerforming final validation...")
note = "Dataset contains kidney tissue samples with chromophobe RCC information in row 7"
if 'is_biased' not in locals():
    is_biased = True

is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available,
    is_biased=is_biased,
    df=linked_data if 'linked_data' in locals() and not linked_data.empty else pd.DataFrame(),
    note=note
)

# 7. Save linked data if usable
if is_usable and 'linked_data' in locals() and not linked_data.empty:
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    
    # Save linked data
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print(f"Dataset not usable for {trait} association studies. Data not saved.")


Normalizing gene symbols...
After normalization: 12700 unique gene symbols


Normalized gene expression data saved to ../../output/preprocess/Kidney_Chromophobe/gene_data/GSE19949.csv

Extracting clinical features...
Clinical data shape: (0, 0)
Clinical data columns: []
Clinical data index: []
Clinical data extracted with shape: (2, 0)
Clinical data preview:
{'Kidney_Chromophobe': [], 'Gender': []}
Clinical data saved to ../../output/preprocess/Kidney_Chromophobe/clinical_data/GSE19949.csv

Linking clinical and genetic data...
Cannot link data: clinical or genetic data is missing/empty

Performing final validation...
Abnormality detected in the cohort: GSE19949. Preprocessing failed.
A new JSON file was created at: ../../output/preprocess/Kidney_Chromophobe/cohort_info.json
Dataset not usable for Kidney_Chromophobe association studies. Data not saved.
